In [1]:
# Load libraries
import os
import email
import pandas as pd


In [2]:
# List of all folders
all_folders=os.listdir("./SpamAssassinMessages")
all_folders.remove(".DS_Store")
all_folders.remove(".ipynb_checkpoints")
all_folders.sort()
all_folders

['easy_ham', 'easy_ham_2', 'hard_ham', 'spam', 'spam_2']

In [3]:
# Count emails
main_="./SpamAssassinMessages/"
count=0
for i in all_folders:
    count+=len(os.listdir(main_+i))
print("Print total emails: ",count)


Print total emails:  9353


+ interesting file ***0352.f7adb4aa267e50a8db1e4bcacfe863f3*** and file ***00341.99b463b92346291f5848137f4a253966*** are the same email

# Parse Data

In [5]:
def parse_html(text_,t):
    if "html" in t:
        soup = BeautifulSoup(text_, "html.parser")
        return(soup.get_text())
    else:
        return(text_)
    

In [8]:
from collections import Counter
from bs4 import BeautifulSoup

types=Counter()
# missed_=[]
msjs=[]
labels=[]
file_names=[]
all_list=Counter()
count=0
notMiss_=0

for root, dirs, files in os.walk("SpamAssassinMessages"):
    for f in files:
        if f != ".DS_Store":
            labels.append(int("spam" in root))
            file_names.append(os.path.join(root,f))
            
            notMiss_=0
            with open(os.path.join(root,f),'r',encoding='latin-1') as file_point:
                count+=1
                msg=email.message_from_file(file_point, )
                type_=msg.get_content_type()
                types[type_]+=1
                # Multipart/report
                if  (type(msg.get_payload())==list) and ("text" in msg.get_payload()[0].get_content_type()):
                    type_=msg.get_payload()[0].get_content_type()
                    msg=msg.get_payload()[0].get_payload()
                    msjs.append(parse_html(msg,type_))
                    notMiss_+=1
                    continue
                if type_=='multipart/signed' or type_=='multipart/mixed' or type_=='multipart/related':
                    # first file is Multi/mixm and within the first file is txt
                    if (msg.get_payload()[0].get_content_type()=="multipart/mixed") or (msg.get_payload()[0].get_content_type()=="multipart/alternative"):
                        type_=msg.get_payload()[0].get_payload()[0].get_content_type()
                        msg=msg.get_payload()[0].get_payload()[0].get_payload()
                        msjs.append(parse_html(msg,type_))
                        notMiss_+=1
                        continue
                if "text" in type_ or type(msg.get_payload())==str:
                    msg=msg.get_payload()
                    msjs.append(parse_html(msg,type_))
                    notMiss_+=1
                if notMiss_==0:
                    all_list[type_]+=1
                    msjs.append(msg)
                    print(file_point)

print(types)


Counter({'text/plain': 7413, 'text/html': 1193, 'multipart/alternative': 326, 'multipart/signed': 180, 'multipart/mixed': 179, 'multipart/related': 56, 'multipart/report': 5, 'text/plain charset=us-ascii': 1})


In [9]:
#Check q
print("Total Emails retrieved: ", len(msjs),'\n Total Emails:',count)


Total Emails retrieved:  9353 
 Total Emails: 9353


In [10]:
#Captures all missed emails
all_list


Counter()

+ Create Data Frame

In [178]:
data=pd.DataFrame({'MessageFile':file_names,"Target":labels, "Emails":msjs})
data.head()

,MessageFile,Target,Emails
0,SpamAssassinMessages/spam/00249.5f45607c1bffe8...,1,"Dear Homeowner,\n \nInterest Rates are at thei..."
1,SpamAssassinMessages/spam/0355.94ebf637e4bd3db...,1,"Friend,Now you can copy DVD's and Games\nhttp:..."
2,SpamAssassinMessages/spam/0395.bb934e8b4c39d5e...,1,Pocket the newest 8 year annuity!\t Pocket th...
3,SpamAssassinMessages/spam/0485.902136727883317...,1,\nCongratulations! You Get a Free Handheld Org...
4,SpamAssassinMessages/spam/00373.ebe8670ac56b04...,1,ATTENTION: This is a MUST for ALL Computer Use...


# Cleaning the data

In [16]:
# # English stop words
# nlp = spacy.load('en_core_web_sm')
# stop_words = nlp.Defaults.stop_words

In [171]:
# Clean up text
import nltk
import numpy as np
import contractions
import re

stop_words= nltk.corpus.stopwords.words('english')
def normalize_document(doc):
    """Function from our class python files to normalize text""" 
    # Expand contractions
    doc=contractions.fix(doc.lower()) 
    # Remove HTLM section
    doc = re.sub(r'https?://\S+|www\.\S+', " ", doc, re.I|re.A)
    # remove special characters\whitespaces
    doc = re.sub(r'[^a-z]', ' ', doc)
#     doc = re.sub(r'[^a-zA-Z]', ' ', doc, re.I|re.A)
    # Remove tags \n \t
    re.sub(r'<[^>]*>|[\n\t]', ' ', doc)
#     doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

#Normalize text
cleanedEmails = normalize_corpus(data["Emails"])


In [172]:
data["Emails"]=cleanedEmails


In [173]:
data.head()


,MessageFile,Target,Emails
0,SpamAssassinMessages/spam/00249.5f45607c1bffe8...,1,dear homeowner interest rates lowest point yea...
1,SpamAssassinMessages/spam/0355.94ebf637e4bd3db...,1,friend copy dvd games backup dvd video cd r bu...
2,SpamAssassinMessages/spam/0395.bb934e8b4c39d5e...,1,pocket newest year annuity pocket newest year ...
3,SpamAssassinMessages/spam/0485.902136727883317...,1,congratulations get free handheld organizer de...
4,SpamAssassinMessages/spam/00373.ebe8670ac56b04...,1,attention must computer users new special pack...


In [150]:
# re.sub(r'https?://\S+|www\.\S+', " ", data["Emails"][30], re.I|re.A)
data["Emails"][30]

'diversified underwriters services inc impaired risk case month july male non smoker face amount lbs crohn disease years major intestinal surgeries steroid therapy years diabetes hypertension diversified answer standard broker commission let us turn clients declined rated current health problems placeable life cases inst quote tm impaired risk life quotes call inst quote client get back within hours ext please fill form information name e mail phone city state broker use public dissemination want anyone receive mailings wish professional communication sent insurance professionals removed mailing list reply message instead go legal notice'

In [174]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data["Emails"])
# Convert the matrix to a Pandas DataFrame
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Add the IDF scores for each token as a new row
df_tfidf.loc['IDF'] = tfidf.idf_


In [190]:
data.head()

,MessageFile,Target,Emails
0,SpamAssassinMessages/spam/00249.5f45607c1bffe8...,1,"Dear Homeowner,\n \nInterest Rates are at thei..."
1,SpamAssassinMessages/spam/0355.94ebf637e4bd3db...,1,"Friend,Now you can copy DVD's and Games\nhttp:..."
2,SpamAssassinMessages/spam/0395.bb934e8b4c39d5e...,1,Pocket the newest 8 year annuity!\t Pocket th...
3,SpamAssassinMessages/spam/0485.902136727883317...,1,\nCongratulations! You Get a Free Handheld Org...
4,SpamAssassinMessages/spam/00373.ebe8670ac56b04...,1,ATTENTION: This is a MUST for ALL Computer Use...


In [192]:
dense_array = tfidf_matrix.toarray()

# Check the shapes
print("Original data shape:", np.array(data).shape)
print("Dense array shape:", dense_array.shape)

Original data shape: (9353, 3)
Dense array shape: (9353, 88984)


In [175]:
len(df_tfidf.columns)

79978

In [182]:
len(english_word_list)

16523

In [183]:
from nltk.corpus import words

# Create a set of English words
english_words = set(words.words())

# Your list of words to filter (replace this with your own list)
word_list = df_tfidf.columns

# Filter non-English words
english_word_list = [word for word in word_list if word.lower() in english_words]

# Print the filtered list of English words
print(english_word_list)


['aa', 'aardvark', 'aba', 'abac', 'abandon', 'abandoned', 'abandonment', 'abatement', 'abb', 'abbas', 'abbey', 'abbreviation', 'abdominal', 'abduction', 'abed', 'aberrant', 'aberration', 'abhorrent', 'abide', 'ability', 'abject', 'able', 'ably', 'abnegate', 'abnormal', 'abnormality', 'abnormally', 'aboard', 'abode', 'abolish', 'abolition', 'aboriginal', 'abort', 'aborted', 'abortion', 'abortive', 'abound', 'abreast', 'abridged', 'abroad', 'abruptly', 'absconded', 'absence', 'absent', 'absinthe', 'absinthium', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorber', 'absorbing', 'absorption', 'absquatulate', 'abstract', 'abstracted', 'abstraction', 'absurd', 'absurdity', 'absurdly', 'abu', 'abundance', 'abundant', 'abundantly', 'abusable', 'abuse', 'abusive', 'abysmal', 'abyss', 'academic', 'academical', 'academy', 'acca', 'accelerate', 'accelerated', 'acceleration', 'accelerator', 'accent', 'accentuate', 'accept', 'acceptability', 'acceptable', 'acceptance', 'accepted', 'access', 'a

In [201]:
df_tfidf=df_tfidf[english_word_list]
df_tfidf.shape

# Run cluster

In [ ]:
from sklearn.cluster import DBSCAN

clustering = DBSCAN(eps=3, min_samples=2).fit(df_tfidf)
clustering.labels_
clustering

In [179]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Initialize the TfidfVectorizer with the desired settings
vectorizer = TfidfVectorizer()

# Compute the document-term matrix of TF-IDF scores
tfidf_matrix = vectorizer.fit_transform(data["Emails"])

# Get the list of tokens and their corresponding IDF scores
tokens = vectorizer.get_feature_names_out()
idfs = vectorizer.idf_

# Print the table of tokens and their corresponding TF-IDF scores
print("{:<15} {:<15}".format("Token", "TF-IDF Score"))
print("-" * 30)
for token, idf in zip(tokens, idfs):
    print("{:<15} {:<15.4f}".format(token, idf))

Token           TF-IDF Score   
------------------------------
00              2.7677         
000             3.3671         
0000            5.9389         
00000           9.4504         
000000          6.2934         
00000000        9.4504         
000000000       9.0449         
00000002        9.4504         
00000004        9.4504         
0000001         9.0449         
0000002         9.0449         
0000004         9.0449         
0000015         9.0449         
0000016         9.0449         
0000018         9.0449         
0000020         9.0449         
0000023         9.0449         
00000254        9.0449         
0000035         9.0449         
0000037         9.0449         
00001           8.1976         
00002           8.3518         
00003           8.3518         
00004           8.3518         
00005           8.3518         
000058          9.0449         
00005d45        9.4504         
00006           8.3518         
000066          8.5341         
00007    

averages        7.2532         
averagesprogramrate30y 9.0449         
averaging       8.3518         
averill         9.0449         
averse          7.8410         
aversion        9.0449         
aversoft        9.0449         
avert           9.4504         
averted         8.5341         
averò           7.5786         
avez            7.7457         
avfs            9.4504         
avg             7.1991         
avgas           8.5341         
avghuh          9.4504         
avi             7.9463         
aviation        7.5045         
aviationmr      9.0449         
aviators        8.5341         
avid            7.3103         
avidemux        9.4504         
avidly          9.4504         
avien           9.4504         
avifile         7.5786         
avifile6        8.5341         
avilable        9.4504         
avinash         9.4504         
avinash1        9.4504         
aving           8.5341         
avings          8.7573         
aviplay         9.0449         
a

exhaustion      7.9463         
exhaustive      7.7457         
exhausts        8.5341         
exhgw8ctk5xrinfrk 9.0449         
exhibit         7.3710         
exhibited       8.7573         
exhibiting      9.0449         
exhibition      7.5786         
exhibitions     8.1976         
exhibitionsindia 9.0449         
exhibitor       9.4504         
exhibitors      8.3518         
exhibits        7.7457         
exhilarateddespair 9.0449         
exhumed         8.5341         
exi             9.0449         
exibitionists   9.0449         
exile           7.4355         
exiled          8.7573         
exim            7.8410         
exim4           9.0449         
exist           4.9961         
existance       9.0449         
existed         6.3149         
existence       6.1546         
existent        7.3103         
existential     8.7573         
existentialist  9.0449         
existentialmoo  9.0449         
existing        4.7545         
existnig        9.4504         
ex

outward         7.5045         
outwardly       9.0449         
outwards        8.5341         
outweighed      8.3518         
outweighs       7.6587         
ouvre           9.0449         
ouvrent         8.5341         
ouwei           8.7573         
ouweilighting   8.7573         
ouwovruzpbpiqw  9.4504         
ov              9.4504         
ov4mrtkljlx     9.0449         
oval            8.1976         
ovals           9.0449         
ovarian         9.0449         
ovative         9.4504         
ove             7.8410         
oven            9.0449         
over            2.8021         
over_           9.0449         
overage         9.0449         
overall         5.7368         
overblown       9.0449         
overbred        9.4504         
overburdened    8.5341         
overbye         9.4504         
overcharging    9.0449         
overclockers    8.5341         
overcocking     9.4504         
overcome        7.1991         
overcoming      8.5341         
overconf

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [45]:
df_tfidf.columns.unique()

Index(['00', '000', '0000', '00000', '000000', '00000000', '000000000',
       '00000002', '00000004', '0000001',
       ...
       'þàïà', 'þàïàº', 'þàïáö¼ò', 'þë', 'þîñ', 'þõµ', 'þö', 'þööööà', 'ÿò',
       'ÿÿã'],
      dtype='object', length=82563)

In [ ]:
##### Vectorize the text with TFIDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

In [746]:

import contractions
import re
def normalized_(text1):
    text1=[words.replace("\n", "") for words in text1] 
    print("Remove \n")
    text1=[contractions.fix(sentence.lower()) for sentence in text1]
    print("Lower_")
    text1=[re.sub(r"[^a-zA-Z'']+", " ", sentence) for sentence in text1]
    print("Remove empty spaces")
    text1=[sentence.strip() for sentence in text1]

    return(text1)


0       Dear Homeowner,\n \nInterest Rates are at thei...
1       Friend,Now you can copy DVD's and Games\nhttp:...
2        Pocket the newest 8 year annuity!\t Pocket th...
3       \nCongratulations! You Get a Free Handheld Org...
4       ATTENTION: This is a MUST for ALL Computer Use...
                              ...                        
9348    I'm one of the 30,000 but it's not working ver...
9349    Damien Morton quoted:\n>W3C approves HTML 4 'e...
9350    On Mon, 2002-07-22 at 06:50, che wrote:\n\n> t...
9351    Once upon a time, Manfred wrote :\n\n> I would...
9352    If you run Pick, and then use the "New FTOC" b...
Name: Emails, Length: 9353, dtype: object